# Detect Sentyment

Nural Network for evaluating sentyment of movie reviews.

Activate the environment.

In [ ]:
using Pkg
Pkg.activate(".")

Load custom library code.

In [ ]:
include("./src/ComputationalGraph.jl")
include("./src/Printing.jl")
include("./src/GraphBuilding.jl")
include("./src/ForwardPass.jl")
include("./src/BackwardPass.jl")
include("./src/ScalarOperations.jl")
include("./src/BroadcastOperations.jl")
include("./src/flux_like_api.jl")

# Nural Network

In [ ]:
using JLD2
X_train = load("./data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("./data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("./data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("./data/imdb_dataset_prepared.jld2", "y_test")
nothing

In [ ]:
input_neurons = size(X_train, 1)
hidden_neurons = 32
output_neurons = 1

batch_size = 64
epochs = 5

In [ ]:
using LinearAlgebra

In [ ]:
binary_cross_entropy_loss(y, ŷ) = mean(Constant(-1.0) .* (y .* log.(ŷ) .+ (Constant(1.0) .- y) .* log.(Constant(1.0) .- ŷ)))

In [ ]:
function loss(model, x, y)
  ŷ = model(x)
  E = binary_cross_entropy_loss(y, ŷ); E.name = "loss"
  return E, ŷ
end

In [ ]:
model = Chain(
  Dense(input_neurons, hidden_neurons, relu),
  Dense(hidden_neurons, output_neurons, σ)
)

y = Variable(zeros(1, batch_size), name="y")
x = Variable(zeros(input_neurons, batch_size), name="x")

otp = setup(Adam(), model)

In [ ]:
using Printf, Statistics
using Flux: DataLoader
dataset = DataLoader((X_train, y_train), batchsize=64, shuffle=true)

In [ ]:
L, ŷ_node = loss(model, x, y)
graph = topological_sort(L)

for epoch in 1:epochs
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    t = @elapsed begin
        for (xb, yb) in dataset
            current_batch_size = size(xb, 2)

            x.output .= xb
            y.output .= yb

            lossValue = forward!(graph)

            for param in trainable(model)
                param.gradient = nothing
            end
            backward!(graph)

            update!(otp, model)

            ŷ = ŷ_node.output
            predictions = ŷ .> 0.5
            targets = y.output .> 0.5
            total_correct += count(predictions .== targets)
            total_loss += lossValue[1] * current_batch_size
            total_samples += current_batch_size
        end
    end

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (loss: %.4f, acc: %.4f)",
        epoch, t, avg_loss, avg_acc))

end